# Battle of the Neighborhoods - Coursera Capstone Project 


##### Author: Aaron E

## Introduction 

#### A local BBQ chain would like to expand to a new city. The client however is worried about compitition in the area of future choice. The prospect city is Toronto Canada. Toronto Canada is a rapidly developing city and provides various business oppourtunities. The city has become a major hub for tourist and businesses alike. Becuase of the growing compitetion in the area the stakeholders would like a detailed review of all the compitetion in the Toronto area and a detailed analysis of the surrounding areas.

## The Data

Based on the clients concern, there are the factors that will influence the clients decission.
- Total number of restaurants in a neighborhood. 
- If there is an already developed and sucessful similar restaurant. We will define sucessfull as being in the top 10 most popular.

We will be using Foursquare API to explore the restaurants in the Toronto area and to explore the most common venues while also using a list of zip codes from a wikipedia page to explore the neighborhoods.

#### First lets explore a few choice boroughs in Toronoto, finding the most popular venues 

In [188]:
# Imported needed libraries
# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


#### Read in the html as a Pandas Dataframe 

In [325]:
#get html from wiki page and create soup object
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
canada_df = df 
canada_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Now we must clean up the data

#### Ignor the cells with bourough not assigned

In [326]:
# Ignore cells with Bourough not assigned
canada_df = canada_df[canada_df["Borough"] != "Not assigned"]

#### Group the data

In [327]:
df_toronto = canada_df.groupby(["Postal Code", "Borough"], as_index=False).agg(','.join)
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Replace mising values in Neighborhood with Borough if Neighborhood is not assigned

In [328]:
df_toronto['Neighbourhood'] = np.where(df_toronto['Neighbourhood'] == 'Not assigned',
                                              df_toronto['Borough'], df_toronto['Neighbourhood'])

#### Lets check the data shape

In [329]:
df_toronto.shape

(103, 3)

#### Lets get the geosatial data and add it to our dataframe

In [330]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)

In [331]:
merged_data = pd.merge(df_toronto, geospatial_data, on='Postal Code')

In [332]:
df_toronto = merged_data

In [333]:
df_toronto.head(100)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
95,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188


## Lets see a map of Toronto with all the neighborhood 

In [33]:
address = "Toronto, Ontario"
geolocator = Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


In [209]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## This client is interested in a more populated part of the city, below we will anaylize Downtown Toronto.

In [36]:
# Credentials for Foursquare API
# @hidden_cell
CLIENT_ID = 'S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5' # your Foursquare ID
CLIENT_SECRET = '4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5
CLIENT_SECRET:4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE


In [211]:
down_town_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
down_town_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [213]:
down_town_data["Neighbourhood"].count()

19

# 19 neighborhoods in Downtown Toronoto

### Map of Downtown Toronto

In [271]:
address_downtown = 'Downtown Toronto'
latitude_downtown = 43.667967
longitude_downtown = -79.3676675
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [272]:
map_downtown = folium.Map(location=[latitude_downtown, longitude_downtown], zoom_start=12)

# add markers to map
for lat, lng, label in zip(down_town_data['Latitude'], down_town_data['Longitude'], down_town_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_downtown)  
    
map_downtown

### Top venues in Downtown Toronto

In [300]:
down_town_data.loc[1, "Neighbourhood"]

'St. James Town, Cabbagetown'

In [301]:
neighbourhood_lat = down_town_data.loc[1,"Latitude"] # Neighbourhood lat value
neighbourhood_long = down_town_data.loc[1, "Longitude"] # Neighbourhood long value
neighbourhood_name = down_town_data.loc[1, "Neighbourhood"] # neighbourhoods name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_lat, 
                                                               neighbourhood_long))

Latitude and longitude values of St. James Town, Cabbagetown are 43.667967, -79.3676753.


#### Setting a radius, data limit, and a search query. First lets looks for all restuarants in the downtown area.

In [302]:
search_query = 'Restaurant'
LIMIT = 500
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5&client_secret=4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE&ll=43.6534817,-79.3839347&v=20180605&query=Restaurant&radius=1000&limit=500'

In [303]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f22ed415b5de7522ff455ec'},
 'response': {'venues': [{'id': '4ad4c05ff964a52048f720e3',
    'name': 'Hemispheres Restaurant & Bistro',
    'location': {'address': '110 Chestnut Street',
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'distance': 224,
     'postalCode': 'M5G 1R3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1596124522',
    'hasPerk': False},
   {'id': '

In [315]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1596124522,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1596124522,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
2,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1596124522,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN
3,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1596124522,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,NaN
4,4bd47e6fcfa7b7139f2924da,Studio Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1596124522,False,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",966,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,NaN


#### Lets keep only columns that include a venue name, and anything that is associated with the chosen location.

In [316]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3
3,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
4,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",966,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,4bd47e6fcfa7b7139f2924da
5,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,"[{'label': 'display', 'lat': 43.64474919591934...",976,M5V 2X3,CA,Toronto,ON,Canada,[225 Front St W (in InterContinental Toronto C...,in InterContinental Toronto Centre,Entertainment District,4b223f5af964a520ba4424e3
6,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",906,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,NaN,4aef8854f964a5201cd921e3
7,North-East Chinese Restaurant 華北美食,Chinese Restaurant,476 Dundas St.,43.653185,-79.396677,"[{'label': 'display', 'lat': 43.6531846832903,...",1026,NaN,CA,Toronto,ON,Canada,"[476 Dundas St. (Huron), Toronto ON, Canada]",Huron,NaN,4d06c1ccc2e53704428cc567
8,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,"[{'label': 'display', 'lat': 43.64563436248102...",1048,M5V 1H3,CA,Toronto,ON,Canada,"[30 Mercer Street (at John St), Toronto ON M5V...",at John St,NaN,4ad4c05df964a5203ff620e3
9,Cottage Restaurant & Lounge,Thai Restaurant,338 Jarvis St.,43.662770,-79.376894,"[{'label': 'display', 'lat': 43.66277006654972...",1179,M4Y 2G6,CA,Toronto,ON,Canada,"[338 Jarvis St. (north of Carlton Street), Tor...",north of Carlton Street,NaN,4ad4c05ff964a52018f720e3


In [317]:
print("List of Resaurants in Downtown Toronto:")
dataframe_filtered.name

List of Resaurants in Downtown Toronto:


0                   Hemispheres Restaurant & Bistro
1                             Victoria's Restaurant
2                    The Hot House Restaurant & Bar
3                Some Time BBQ Grill Restaurant 碳烤屋
4                                 Studio Restaurant
5                            Azure Restaurant & Bar
6                      Aroma Fine Indian Restaurant
7                North-East Chinese Restaurant 華北美食
8                           Victor Restaurant & Bar
9                       Cottage Restaurant & Lounge
10                                   360 Restaurant
11                            Restaurant | Adelaide
12                                Tundra Restaurant
13                          Osgoode Hall Restaurant
14              Richtree Natural Market Restaurants
15                           The Senator Restaurant
16                                  Cali Restaurant
17                  Kyoto House Japanese Restaurant
18    Spring Rolls | Japanese Restaurant in Toronto
19          

In [318]:
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3
3,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
4,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",966,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,4bd47e6fcfa7b7139f2924da


### 49 Restauarnts have been identified in Downtown Toronoto

### Next lets visualize these establishments to get more of a sense of location 

In [246]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### The client is also interested in the area, so lets see what other types of venues are offered.

In [247]:
downtown_neighborhood_venues = getNearbyVenues(names=down_town_data['Neighbourhood'],
                                   latitudes=down_town_data['Latitude'],
                                   longitudes=down_town_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [248]:
downtown_neighborhood_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
5,"St. James Town, Cabbagetown",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
6,"St. James Town, Cabbagetown",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
7,"St. James Town, Cabbagetown",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
8,"St. James Town, Cabbagetown",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
9,"St. James Town, Cabbagetown",43.667967,-79.367675,Merryberry Cafe + Bistro,43.666630,-79.368792,Café


In [249]:
downtown_neighborhood_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [204]:
print('There are {} uniques categories.'.format(len(downtown_neighborhood_venues['Venue Category'].unique())))

There are 210 uniques categories.


# Count of how many venues are in each neighborhood; there are 209 unique categories in Downtown Toronto

# Next we will see the type of venues in each neighborhood in Downtown Toronoto

In [250]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1238,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1239,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1240,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouped neighborhoods by mean of frewuency of ocurrence of each category 

In [251]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0.000806,0.000806,0.000806,0.000806,0.001613,0.002419,0.001613,0.01371,0.001613,...,0.000806,0.000806,0.002419,0.01129,0.000806,0.003226,0.006452,0.000806,0.000806,0.005645
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


SyntaxError: EOL while scanning string literal (<ipython-input-280-0414d6ff791a>, line 10)

# Lets simplify and find the top 10 venues 

In [252]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [253]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Gym,Bakery,Seafood Restaurant
1,1,Yoga Studio,Gym / Fitness Center,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


# Results for Downtown Toronto

#### The area of Downtown Toronto consit of 49 established restaurants.
#### listed here:                 
-                    Hemispheres Restaurant & Bistro
-                              Victoria's Restaurant
-                     The Hot House Restaurant & Bar
-                 Some Time BBQ Grill Restaurant 碳烤屋
-                             Azure Restaurant & Bar
-            Tasty's Caribbean Restaurant & Catering
-                                  Studio Restaurant
-                            Victor Restaurant & Bar
-                             Rol San Restaurant 龍笙棧
-                 North-East Chinese Restaurant 華北美食
-                     Green Tea Restaurant Downtown
-                       Cottage Restaurant & Lounge
-                    Goldstone Noodle Restaurant 金石
-                    Aroma Fine Indian Restaurant
-                           Sightlines Restaurant
-                Kensington Cornerstone Restaurant
-                            Restaurant | Adelaide
-               Sky Dragon Chinese Restaurant 龍翔酒樓
-   Sassafraz | Cafe | Restaurant | Private Events
-                        Ka Chi Korean Restaurant
-                                Tundra Restaurant
-                    Micheal's Restaurant and Deli
-                         New Sky Restaurant 小沙田食家
-                            ONE Restaurant/Lounge
-                       Matisse Restaurant And Bar
-                                   360 Restaurant
-                          Swatow Restaurant 汕頭小食家
-                          Osgoode Hall Restaurant
-                                 Union Restaurant
-             Richtree Natural Market Restaurants
-                           The Senator Restaurant
-                                   Cali Restaurant
-                Castries Bar & Five Way Restaurant
-                                  Maezo Restaurant
-     Spring Rolls | Japanese Restaurant in Toronto
-                            Tender Trap Restaurant
-                     Terrace Restaurant at Elmwood
-                            Maezo Restaurant & Bar
-                   Mapo Korean BBQ Restaurant 마포상회
-                 El Rancho Restaurant & Night Club
-                    Insomnia Restaurant and Lounge
-                        Alio Restaurant & Wine Bar
-                     Yulla Asian Fusion Restaurant
-                              Donatello Restaurant
-                   Kyoto House Japanese Restaurant
-                     Hong Shing Chinese Restaurant
-                           New Treasure Restaurant
-                                 Anoush Restaurant
-                        Hendricks Restaurant & Bar
-                      Vineyard Jamaican Restaurant

#### None of wich are BBQ restaurants other than "Mapo Korean BBQ", but this is a different category of "BBQ restaurant". The 10 ten venues in each neighborhood do consit of 7 restauants, indecating that the people in this area do enjoy eating out. None of the top 10 venues are a BBQ restuant, and there appears to be no indecation of an established BBQ restaurant, giving the opportunity to be the first on in the area.

# The most popular venue is a coffee shop in Rosedale, and a Yoga Studio in Jamestown/Cabbage Town.

# Lets now explore another area

In [344]:
# Credentials
# @hidden_cell
CLIENT_ID = 'S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5' # your Foursquare ID
CLIENT_SECRET = '4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5
CLIENT_SECRET:4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE


In [345]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Map of Scarborough

In [346]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [347]:
map_scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

### Top venues in the neighborhood "Scarborough Village"

In [348]:
scarborough_data.loc[5, "Neighbourhood"]

'Scarborough Village'

In [340]:
address = "Toronto, Scarborough"
geolocator = Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.773077, -79.257774.


In [349]:
neighbourhood_lat = scarborough_data.loc[5,"Latitude"] # Neighbourhood lat value
neighbourhood_long = scarborough_data.loc[5, "Longitude"] # Neighbourhood long value
neighbourhood_name = scarborough_data.loc[5, "Neighbourhood"] # neighbourhoods name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_lat, 
                                                               neighbourhood_long))

Latitude and longitude values of Scarborough Village are 43.7447342, -79.23947609999999.


In [373]:
search_query = 'Restaurant'
LIMIT = 500
radius = 4000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5&client_secret=4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE&ll=43.773077,-79.257774&v=20180605&query=Restaurant&radius=4000&limit=500'

In [374]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f22f55c85d1da39f9a674c0'},
 'response': {'venues': [{'id': '4b29e021f964a520d4a324e3',
    'name': 'Perfect Chinese Restaurant 雅瓊海鮮酒家',
    'location': {'address': '4386 Sheppard Ave. E',
     'crossStreet': 'at Brimley Rd.',
     'lat': 43.78777423062292,
     'lng': -79.27029393705004,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.78777423062292,
       'lng': -79.27029393705004}],
     'distance': 1920,
     'postalCode': 'M1S 1T8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['4386 Sheppard Ave. E (at Brimley Rd.)',
      'Toronto ON M1S 1T8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referral

In [375]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4b29e021f964a520d4a324e3,Perfect Chinese Restaurant 雅瓊海鮮酒家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1596126636,False,4386 Sheppard Ave. E,at Brimley Rd.,43.787774,-79.270294,"[{'label': 'display', 'lat': 43.78777423062292...",1920,M1S 1T8,CA,Toronto,ON,Canada,"[4386 Sheppard Ave. E (at Brimley Rd.), Toront...",NaN,NaN
1,4d570727fb65236a7f600db4,Silver Spoon Pak-Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1596126636,False,"4800 Sheppard Avenue East, Unit 124-126",NaN,43.791930,-79.251253,"[{'label': 'display', 'lat': 43.79192991300102...",2163,M1S 4N5,CA,Toronto,ON,Canada,"[4800 Sheppard Avenue East, Unit 124-126, Toro...",Agincourt,NaN
2,4bf96c435317a593a23a017f,Karaikudi Chettinad South Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1596126636,False,1225 Kennedy Rd,at Forbes (Between Lawrence and Ellesmere),43.756042,-79.276276,"[{'label': 'display', 'lat': 43.75604153945313...",2410,M1P 4Y1,CA,Toronto,ON,Canada,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,NaN,33638214
3,53dbb77e498e6dfadda043fa,Old Neighbour Restaurant 老街坊天津韩记包子铺,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1596126636,False,"25 Glen Watford Dr, Unit 9",at Sheppard Ave E,43.787078,-79.275454,"[{'label': 'display', 'lat': 43.78707762430626...",2109,M1S 2B7,CA,Toronto,ON,Canada,"[25 Glen Watford Dr, Unit 9 (at Sheppard Ave E...",NaN,NaN
4,4be303c52fc7d13ae879083a,Beef Noodle Restaurant 老李牛肉麵,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1596126636,False,4271 Sheppard Ave. E,btwn Brimley & Midland Ave.,43.785937,-79.276031,"[{'label': 'display', 'lat': 43.78593747713555...",2050,M1S 4G4,CA,Scarborough,ON,Canada,[4271 Sheppard Ave. E (btwn Brimley & Midland ...,NaN,NaN


In [376]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Perfect Chinese Restaurant 雅瓊海鮮酒家,Chinese Restaurant,4386 Sheppard Ave. E,at Brimley Rd.,43.787774,-79.270294,"[{'label': 'display', 'lat': 43.78777423062292...",1920,M1S 1T8,CA,Toronto,ON,Canada,"[4386 Sheppard Ave. E (at Brimley Rd.), Toront...",NaN,4b29e021f964a520d4a324e3
1,Silver Spoon Pak-Indian Restaurant,Indian Restaurant,"4800 Sheppard Avenue East, Unit 124-126",NaN,43.791930,-79.251253,"[{'label': 'display', 'lat': 43.79192991300102...",2163,M1S 4N5,CA,Toronto,ON,Canada,"[4800 Sheppard Avenue East, Unit 124-126, Toro...",Agincourt,4d570727fb65236a7f600db4
2,Karaikudi Chettinad South Indian Restaurant,Indian Restaurant,1225 Kennedy Rd,at Forbes (Between Lawrence and Ellesmere),43.756042,-79.276276,"[{'label': 'display', 'lat': 43.75604153945313...",2410,M1P 4Y1,CA,Toronto,ON,Canada,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,NaN,4bf96c435317a593a23a017f
3,Old Neighbour Restaurant 老街坊天津韩记包子铺,Chinese Restaurant,"25 Glen Watford Dr, Unit 9",at Sheppard Ave E,43.787078,-79.275454,"[{'label': 'display', 'lat': 43.78707762430626...",2109,M1S 2B7,CA,Toronto,ON,Canada,"[25 Glen Watford Dr, Unit 9 (at Sheppard Ave E...",NaN,53dbb77e498e6dfadda043fa
4,Beef Noodle Restaurant 老李牛肉麵,Chinese Restaurant,4271 Sheppard Ave. E,btwn Brimley & Midland Ave.,43.785937,-79.276031,"[{'label': 'display', 'lat': 43.78593747713555...",2050,M1S 4G4,CA,Scarborough,ON,Canada,[4271 Sheppard Ave. E (btwn Brimley & Midland ...,NaN,4be303c52fc7d13ae879083a


In [377]:
print("List of Resaurants in Downtown Toronto:")
dataframe_filtered.name.count()

List of Resaurants in Downtown Toronto:


0                   Perfect Chinese Restaurant 雅瓊海鮮酒家
1                  Silver Spoon Pak-Indian Restaurant
2         Karaikudi Chettinad South Indian Restaurant
3                 Old Neighbour Restaurant 老街坊天津韩记包子铺
4                        Beef Noodle Restaurant 老李牛肉麵
5                 Best Friends Chinese Restaurant 會賓樓
6           South Sea Fish Village Chinese Restaurant
7                          Sagano Japanese Restaurant
8     In Cheon House Korean & Japanese Restaurant 인천관
9                           Sammy's Family Restaurant
10                           Fortune House Restaurant
11                   Very Fair Chinese Restaurant 同德樓
12                          Wok Wok Restaurant 恒記粥麵小廚
13                            JXY Dumpling Restaurant
14                         Supreme Restaurant and Bar
15                         Starlite Family Restaurant
16             Chopsticks SzeChuan Chinese Restaurant
17            Golden Seven Chinese Cuisine Restaurant
18                 The Royal

### Again lets visualize these established resturants in Scarborough

In [378]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [366]:
scarborough_nieghnorhood_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [367]:
scarborough_nieghnorhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [368]:
scarborough_nieghnorhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",8,8,8,8,8,8
"Golden Mile, Clairlea, Oakridge",8,8,8,8,8,8
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",7,7,7,7,7,7


In [369]:
print('There are {} uniques categories.'.format(len(scarborough_nieghnorhood_venues['Venue Category'].unique())))

There are 55 uniques categories.


# There are 55 unique categories in the borough of Scarborough

In [370]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_nieghnorhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_nieghnorhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,...,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


grouped neighborhoods by mean of frequency of occurrence of each category

In [371]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,...,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.25,0.000,0.0,0.000,...,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.000,0.000000,0.000
1,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.0,0.000,...,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.000,0.000000,0.000
2,Cedarbrae,0.0,0.111111,0.111111,0.111111,0.0,0.00,0.000,0.0,0.000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.111111,0.000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000000,0.000000,0.071429,0.0,0.00,0.000,0.0,0.000,...,0.142857,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.071429,0.000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.0,0.000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.000000,0.000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000000,0.000000,0.000000,0.0,0.00,0.125,0.0,0.000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.000000,0.125
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000000,0.250000,0.000000,0.0,0.00,0.000,0.0,0.125,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.125,0.000000,0.000


### top 10 venues per neighboring in Scarborough

In [372]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Golf Course,Gas Station,Furniture / Home Store,Fried Chicken Joint
2,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Thai Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Department Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pharmacy,Pizza Place,Gas Station,Intersection,Convenience Store,Noodle House,Italian Restaurant,Fast Food Restaurant,Fried Chicken Joint,Bank
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Athletics & Sports,Bakery,Golf Course,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Brewery,Furniture / Home Store,Light Rail Station,Pet Store,Chinese Restaurant,Breakfast Spot,Bubble Tea Shop,General Entertainment
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Station,Soccer Field,Ice Cream Shop,Metro Station,Park,Bus Line,Vietnamese Restaurant,Convenience Store,Department Store
7,"Guildwood, Morningside, West Hill",Breakfast Spot,Restaurant,Electronics Store,Intersection,Medical Center,Rental Car Location,Mexican Restaurant,Bank,Fried Chicken Joint,Fast Food Restaurant
8,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Bus Station,Discount Store,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Brewery,Bubble Tea Shop,Golf Course
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Golf Course,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store


# Results for Scarborough, Canada. 

#### As before are result indecate that there are 50 restaurants in Scarborough Canada
#### listed below:
-                 Perfect Chinese Restaurant 雅瓊海鮮酒家
-                 Silver Spoon Pak-Indian Restaurant
-         Karaikudi Chettinad South Indian Restaurant
-                 Old Neighbour Restaurant 老街坊天津韩记包子铺
-                        Beef Noodle Restaurant 老李牛肉麵
-                 Best Friends Chinese Restaurant 會賓樓
-           South Sea Fish Village Chinese Restaurant
-                          Sagano Japanese Restaurant
-     In Cheon House Korean & Japanese Restaurant 인천관
-                           Sammy's Family Restaurant
-                           Fortune House Restaurant
-                   Very Fair Chinese Restaurant 同德樓
-                          Wok Wok Restaurant 恒記粥麵小廚
-                            JXY Dumpling Restaurant
-                         Supreme Restaurant and Bar
-                         Starlite Family Restaurant
-             Chopsticks SzeChuan Chinese Restaurant
-            Golden Seven Chinese Cuisine Restaurant
-                 The Royal Chinese Restaurant 避風塘小炒
-                      Konner Chinese Restaurant 玉樓東
-                                 Terry's Restaurant
-                           Terry's Restaurant & Bar
-                                      Restaurant 59
-                                 terry's restaurant
-                            La Mere Restaurant 老媽肉餅
-            Jesse Jr. (Filipino Foods & Restaurant)
-                                Federick Restaurant
-                    Joey Bravo's Italian Restaurant
-                             LHL Seafood Restaurant
-                                 Timehri Restaurant
-                                  Susur Restaurant
-                   Yummy Cantonese Restaurant 老西関腸粉
-                               Motimahal Restaurant
-                          The Local Cafe Restaurant
-                          Yau's Lung Kee Restaurant
-                    Ocean Castle Seafood Restaurant
-        Fortune Dynasty Chinese Seafood Restaurant
-                  May Yan Seafood Restaurant 陸福海鮮酒家
-        Continental Soiree Family Restaurant & Bar
-                             Lucky House Restaurant
-                   Twilight Family Restaurant & Bar
-                                 Ox Land Restaurant
-                              pengfuxuan Restaurant
-               Scarborough Grand Seafood Restaurant
-                      King Huang Chinese Restaurant
-                     Inchen House Korean Restaurant
-                          Woburn Garden Restaurant
-                             Grand Ocean Restaurant
-                     Yuan Yuan Chinese Restaurant
-                                 Bendale Restaurant

#### Here there is a more of a mix of top venues, but "Fried Chicken Joint" appearing the most often. There is no indecation of an American BBQ restaurant either again offering the opportinuty of being the first in the area.